In [ ]:
pip install torch torchvision

In [ ]:
import torch
import cv2

In [ ]:
import numpy as np
import pandas as pd
import os
from PIL import Image 
from torch.utils.data import Dataset 
from torchvision import datasets, transforms, models
import torchvision.transforms as transforms # A module from the torchvision library that provides common image transformations, such as resizing, cropping, and normalization.
from torch.utils.data import random_split # A function from PyTorch that allows for randomly splitting a dataset into training and validation subsets.
#from torch.optim.lr_scheduler import ReduceLROnPlateau # A PyTorch scheduler that adjusts the learning rate during training based on a specified metric, reducing it when the metric plateaus.
import torch.nn as nn # A module in PyTorch that provides classes for defining and building neural networks.
from torchvision import utils # A module from torchvision that contains utility functions for working with images, such as saving and visualizing them.
from torch.utils.data import Dataset
from PIL import Image
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
import torch.optim as optim
#import splitfolders

#import torch.nn.functional as F
import pathlib
from sklearn.metrics import confusion_matrix, classification_report
import itertools # This import statement imports the itertools module, which provides functions for efficient looping and combining of iterables. It can be used for tasks such as generating combinations or permutations of elements.
from tqdm.notebook import trange, tqdm # These functions allow for the creation of progress bars to track the progress of loops or tasks.
from torch import optim
import warnings
warnings.filterwarnings('ignore')


In [ ]:
pip install opencv-python

In [ ]:
! pip install wget nibabel # lecture files format .nii



In [ ]:
# ask user to do the commands in the cells in both researcher and node environments

!git clone https://github.com/ultralytics/yolov5  # clone

%cd yolov5
%pip install -qr requirements.txt comet_ml  # install

In [ ]:
model = torch.hub.load('ultralytics/yolov5', 'yolov5s')

torch.save(model.state_dict(), 'mymodel.pt')

In [ ]:
model.eval()

In [ ]:
from models.yolo import Model

architecture = Model(cfg='models/yolov5s.yaml')

In [ ]:
architecture.forward

In [ ]:
import torch.nn as nn

classifier_layer = nn.Sequential(nn.Conv3d(16, 2, kernel_size=(1, 1, 1), stride=(1, 1, 1)))

# Edit your training plan

In [ ]:
from fedbiomed.common.training_plans import TorchTrainingPlan
from flamby.datasets.fed_ixi import Baseline, BaselineLoss, Optimizer
from fedbiomed.common.data import FlambyDataset, DataManager
from models.yolo import Model

class MyTrainingPlan(TorchTrainingPlan):

    
    def init_model(self, model_args):
        return Model(cfg='models/yolov5s.yaml')

    def init_optimizer(self, optimizer_args):
        return Optimizer(self.model().parameters(), lr=optimizer_args["lr"])

    def init_dependencies(self):
        return ["from flamby.datasets.fed_ixi import Baseline, BaselineLoss, Optimizer",
                "from fedbiomed.common.data import FlambyDataset, DataManager",
               "from models.yolo import Model"]

    def training_step(self, data, target):
        output = self.model().forward(data)
        return BaselineLoss().forward(output, target)

    def training_data(self):
        dataset = FlambyDataset()
        loader_arguments = { 'shuffle': True}
        return DataManager(dataset, **loader_arguments)

In [ ]:
from flamby.datasets.fed_ixi import Baseline, BaselineLoss


In [ ]:
tp = MyTrainingPlan()

tp.import_model('mymodel.pt')

In [ ]:


model_args = {}

training_args = {
    'loader_args': { 'batch_size': 8, },
    'optimizer_args': {
        "lr" : 1e-3
    },
    'epochs': 1,
    'dry_run': False,
    'batch_maxnum': 2 # Fast pass for development : only use ( batch_maxnum * batch_size ) samples
}


In [ ]:
from fedbiomed.researcher.experiment import Experiment
from fedbiomed.researcher.aggregators.fedavg import FedAverage

tags =  ['ixi']
rounds = 1

exp = Experiment(tags=tags,
                 model_args=model_args,
                 training_plan_class=MyTrainingPlan,
                 training_args=training_args,
                 round_limit=rounds,
                 aggregator=FedAverage(),
                 node_selection_strategy=None)

In [ ]:
exp.training_plan().import_model('mymodel.pt')

In [ ]:
exp.run()

In [ ]:
Baseline().classifier


In [ ]:
Baseline()